In [1]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
def assertFloat(x):
    assert type(float(x)) == float

def assertFloatList(items, N):
    assert len(items) == N
    assert [type(float(x)) for x in items] == [float]*N

In [4]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [5]:
def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        u,b,r = l.strip().split(',')
        r = int(r)
        yield u,b,r

In [6]:
answers = {}

In [7]:
# Some data structures that will be useful

In [233]:
allRatings = []
for l in readCSV("train_Interactions.csv.gz"):
    allRatings.append(l)

In [9]:
len(allRatings)

200000

In [10]:
ratingsTrain = allRatings[:190000]
ratingsValid = allRatings[190000:]
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
for u,b,r in ratingsTrain:
    ratingsPerUser[u].append((b,r))
    ratingsPerItem[b].append((u,r))

In [11]:
##################################################
# Read prediction                                #
##################################################

In [ ]:
ratingsValid[:10]

[('u93397390', 'b52690052', 3),
 ('u93952353', 'b52355478', 1),
 ('u09633433', 'b89163374', 0),
 ('u91366781', 'b78391921', 5),
 ('u50251394', 'b80097453', 0),
 ('u98333620', 'b80430178', 3),
 ('u77448169', 'b59839746', 5),
 ('u26756934', 'b97033148', 5),
 ('u05133116', 'b80332084', 4),
 ('u83823213', 'b21044596', 0)]

In [15]:
# Copied from baseline code
bookCount = defaultdict(int)
totalRead = 0

for user,book,_ in readCSV("train_Interactions.csv.gz"):
    bookCount[book] += 1
    totalRead += 1

mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalRead/2: break

In [16]:
### Question 1

In [413]:
newRatingsValid = [(user, book, 1) for user, book, _ in ratingsValid]
for user, book, _ in ratingsValid:
    newBook = random.choice(list(bookCount.keys()))
    while newBook in [b for b, _ in ratingsPerUser[user]]:
        newBook = random.choice(list(bookCount.keys()))
    newRatingsValid.append((user, newBook, 0))

In [415]:
random.shuffle(newRatingsValid)
newRatingsValid[-10:]

[('u93484583', 'b50378244', 1),
 ('u68538807', 'b32561157', 1),
 ('u89150296', 'b11225717', 1),
 ('u70320476', 'b91966629', 0),
 ('u01232407', 'b80237188', 1),
 ('u41238833', 'b08746938', 0),
 ('u79880271', 'b08876271', 1),
 ('u10811505', 'b12927878', 0),
 ('u04571321', 'b31893168', 1),
 ('u11995627', 'b52448841', 0)]

In [416]:
correct = 0
for _, book, predict in newRatingsValid:
    correct += predict == (book in return1)

acc1 = correct / len(newRatingsValid)
print(acc1)

0.71445


In [417]:
answers['Q1'] = acc1

In [418]:
assertFloat(answers['Q1'])

In [ ]:
### Question 2

In [419]:
def solve(threshold):
    bookCount = defaultdict(int)
    totalRead = 0

    for _ ,book,_ in readCSV("train_Interactions.csv.gz"):
        bookCount[book] += 1
        totalRead += 1

    mostPopular = [(bookCount[x], x) for x in bookCount]
    mostPopular.sort()
    mostPopular.reverse()

    return2 = set()
    count = 0

    for ic, i in mostPopular:
        count += ic
        return2.add(i)
        if count > totalRead * threshold: break

    correct = 0
    for _, book, predict in newRatingsValid:
        correct += predict == (book in return2)
    #print(correct, return2)
    acc = correct / len(newRatingsValid)
    return acc

In [420]:
l, r, eps = 0, 1, 1e-3
while r - l > eps:
    m1 = l + (r - l) / 3
    m2 = r - (r - l) / 3
    print(m1, m2)
    if solve(m1) < solve(m2):
        l = m1
    else:
        r = m2

best_threshold = l



0.3333333333333333 0.6666666666666667
0.5555555555555556 0.7777777777777778
0.7037037037037037 0.8518518518518519
0.654320987654321 0.7530864197530864
0.720164609053498 0.7860082304526749
0.6982167352537723 0.7421124828532236
0.6835848193872885 0.7128486511202561
0.7030940405426003 0.7226032616979119
0.696590966824163 0.7095971142610374
0.7052617317820793 0.7139324967399956
0.7110422417540235 0.7168227517259677
0.7091154050967088 0.7129690784113382
0.7078308473251655 0.7103999628682519
0.7069744754774702 0.708687219172861
0.7081163046077308 0.7092581337379913
0.7077356948976439 0.7084969143178176
0.708243174511093 0.7087506541245422
0.7080740146399433 0.7084123343822427


In [421]:
acc2 = solve(best_threshold)
acc2

0.757

In [422]:
answers['Q2'] = [best_threshold, acc2]

In [423]:
assertFloat(answers['Q2'][0])
assertFloat(answers['Q2'][1])

In [424]:
### Question 3/4

In [425]:
def Jaccard(s1, s2):
    return len(s1.intersection(s2)) / len(s1.union(s2)) 

In [426]:
usersPerItem = defaultdict(set)
itemsPerUser = defaultdict(set)
for user, item, _ in ratingsTrain:
    usersPerItem[item].add(user)
    itemsPerUser[user].add(item)

def solveQ3(threshold):
    ans = set()
    for user, book, _ in newRatingsValid:
        B = usersPerItem[book] #all users who read the book
        maxSim = 0
        for bprime in itemsPerUser[user]:
            if bprime == book:
                continue
            B2 = usersPerItem[bprime]
            sim = Jaccard(B, B2)
            maxSim = max(maxSim, sim)
            #print(maxSim)
        if maxSim > threshold:
            ans.add(book)

    correct = 0
    for _, book, predict in newRatingsValid:
        correct += predict == (book in ans)
    
    acc = correct / len(newRatingsValid)
    return acc


In [440]:
l, r, eps = 0.006, 0.009, 1e-6
while r - l > eps:
    m1 = l + (r - l) / 3
    m2 = r - (r - l) / 3
    now1 = solveQ3(m1)
    now2 = solveQ3(m2)
    print(now1, l, now2, r)
    if now1 < now2:
        l = m1
    else:
        r = m2

best_threshold2 = l

0.69805 0.006 0.7005 0.009
0.69965 0.007 0.6999 0.009
0.70025 0.007666666666666666 0.69865 0.009
0.70045 0.007666666666666666 0.69975 0.008555555555555556
0.6998 0.007666666666666666 0.7005 0.00825925925925926
0.70065 0.007864197530864198 0.70025 0.00825925925925926
0.70045 0.007864197530864198 0.7005 0.008127572016460905
0.7005 0.007951989026063101 0.7004 0.008127572016460905
0.70065 0.007951989026063101 0.7005 0.008069044352994971
0.70065 0.007951989026063101 0.7005 0.008030025910684347
0.70065 0.007951989026063101 0.70065 0.008004013615810598
0.70045 0.007951989026063101 0.70065 0.007986672085894765
0.70065 0.00796355004600699 0.70065 0.007986672085894765
0.70065 0.00796355004600699 0.70065 0.007978964739265506
0.70045 0.00796355004600699 0.70065 0.007973826508179334
0.70065 0.007966975533397772 0.70065 0.007973826508179334
0.70065 0.007966975533397772 0.70065 0.007971542849918812
0.70045 0.007966975533397772 0.70065 0.007970020411078466
0.70065 0.00796799049262467 0.70065 0.0079700

In [445]:
acc3 = solveQ3(best_threshold2)
acc3

0.70045

# --------------------------------- problem 4 -----------------------------------

In [443]:
def solveQ4(threshold1, threshold2):
    ans1 = set()
    for user, book, _ in newRatingsValid:
        B = usersPerItem[book] #all users who read the book
        maxSim = 0
        for bprime in itemsPerUser[user]:
            if bprime == book:
                continue
            B2 = usersPerItem[bprime]
            sim = Jaccard(B, B2)
            maxSim = max(maxSim, sim)
            #print(maxSim)
        if maxSim > threshold1:
            ans1.add(book)

    bookCount = defaultdict(int)
    totalRead = 0

    for user,book,_ in readCSV("train_Interactions.csv.gz"):
        bookCount[book] += 1
        totalRead += 1

    mostPopular = [(bookCount[x], x) for x in bookCount]
    mostPopular.sort()
    mostPopular.reverse()
    count = 0
    ans2 = set()
    for ic, i in mostPopular:
        count += ic
        ans2.add(i)
        if count > totalRead * threshold2: break

    global ret
    ret = set.union(ans1, ans2)
    correct = 0
    for _, book, predict in newRatingsValid:
        correct += predict == (book in ret)
    
    acc = correct / len(newRatingsValid)
    return acc

#idea is u just use 2 threshold instead of 1 for the function and find the maximum 
    


In [359]:
best_threshold3 = scipy.optimize.fmin(lambda x: -solveQ4(x[0], x[1]), [0.699 ,0.6835848193872885 ])

Optimization terminated successfully.
         Current function value: -0.755950
         Iterations: 22
         Function evaluations: 58


In [360]:
best_threshold3

array([0.70391484, 0.70815115])

In [361]:
acc4 = solveQ4(best_threshold3[0], best_threshold3[1])

In [362]:
acc4

0.75595

In [446]:
answers['Q3'] = acc3
answers['Q4'] = acc4

In [447]:
assertFloat(answers['Q3'])
assertFloat(answers['Q4'])

In [365]:
predictions = open("predictions_Read.csv", 'w')
for l in open("pairs_Read.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,b = l.strip().split(',')
    if b in ret:
        predictions.write(u + ',' + b + ",1\n")
    else:
        predictions.write(u + ',' + b + ",0\n")


predictions.close()

In [366]:
answers['Q5'] = "I confirm that I have uploaded an assignment submission to gradescope"

In [367]:
assert type(answers['Q5']) == str

In [ ]:
##################################################
# Rating prediction                              #
##################################################

In [ ]:
### Question 6

In [281]:
    
alpha = numpy.mean([r for _, _, r in ratingsTrain])
beta_user = defaultdict(float)
beta_book = defaultdict(float)
#referenced formula from 10/22 slides.  

for _ in range(50):
    alpha = sum(rating - (beta_user[user] + beta_book[book]) for user, book, rating in ratingsTrain) / len(ratingsTrain)
    for user, items in ratingsPerUser.items():
        beta_user[user] = sum(rating - (alpha + beta_book[book]) for book, rating in items) / (1 + len(items))
    for book, items in ratingsPerItem.items():
        beta_book[book] = sum(rating - (alpha + beta_user[user]) for user, rating in items) / (1 + len(items))

valid_error = [(rating - (alpha + beta_user.get(user, 0) + beta_book.get(book, 0))) ** 2 for user, book, rating in ratingsValid]
validMSE = numpy.mean(valid_error).item()

         

In [ ]:
validMSE

1.4618057928245451

In [264]:
answers['Q6'] = validMSE

In [265]:
assertFloat(answers['Q6'])

In [ ]:
### Question 7

In [277]:
maxUser = str(max(beta_user, key=beta_user.get))
maxBeta = beta_user[maxUser]
minUser = str(min(beta_user, key=beta_user.get))
minBeta = beta_user[minUser]


In [278]:
answers['Q7'] = [maxUser, minUser, maxBeta, minBeta]
answers['Q7']

['u18223169', 'u88024921', 1.8189836349785167, -3.5630493778377224]

In [279]:
assert [type(x) for x in answers['Q7']] == [str, str, float, float]

In [280]:
### Question 8

In [285]:
def solve(lambda_reg):
    alpha = numpy.mean([r for _, _, r in ratingsTrain])
    beta_user = defaultdict(float)
    beta_book = defaultdict(float)
    #referenced formula from 10/22 slides.  

    for _ in range(10):
        alpha = sum(rating - (beta_user[user] + beta_book[book]) for user, book, rating in ratingsTrain) / len(ratingsTrain)
        for user, items in ratingsPerUser.items():
            beta_user[user] = sum(rating - (alpha + beta_book[book]) for book, rating in items) / (lambda_reg + len(items))
        for book, items in ratingsPerItem.items():
            beta_book[book] = sum(rating - (alpha + beta_user[user]) for user, rating in items) / (lambda_reg + len(items))

    valid_error = [(rating - (alpha + beta_user.get(user, 0) + beta_book.get(book, 0))) ** 2 for user, book, rating in ratingsValid]
    validMSE = numpy.mean(valid_error).item()
    return validMSE

In [ ]:
l, r, eps = 0, 100, 1e-4

while(r-l > eps):
    m1 = l + (r-l)/3
    m2 = r - (r-l)/3
    print(m1, m2)
    if solve(m1) < solve(m2):
        r = m2
    else:
        l = m1

print(l, solve(l))

lamb = l
validMSE = solve(l)

33.333333333333336 66.66666666666666
22.222222222222218 44.44444444444444
14.814814814814815 29.629629629629626
9.876543209876543 19.75308641975308
6.584362139917694 13.168724279835388
4.38957475994513 8.779149519890257
2.9263831732967525 5.852766346593505
4.8773052888279205 6.828227404359089
4.226997916984198 5.527612660671644
3.793459669088383 4.660536164880013
4.371510666282803 4.9495616634772235
4.17882700055133 4.564194332014276
4.050371223397014 4.307282777705645
4.221645592936102 4.3929199624751885
4.3358285059621595 4.450011418988218
4.297767534953474 4.373889476970846
4.348515496298389 4.399263457643303
4.331599509183417 4.36543148341336
4.320322184440102 4.342876833926732
4.335358617431188 4.350395050422274
4.3303464731008265 4.34037076176155
4.337029332207976 4.343712191315125
4.341484571612742 4.345939811017508
4.339999491811153 4.342969651414331
4.341979598213271 4.34395970461539
4.341319562745899 4.342639633680644
4.342199610035729 4.34307965732556
4.341906260939119 4.342

In [293]:
answers['Q8'] = (lamb, validMSE)

In [294]:
assertFloat(answers['Q8'][0])
assertFloat(answers['Q8'][1])

In [1]:
predictions = open("predictions_Rating.csv", 'w')
for l in open("pairs_Rating.csv"):
    if l.startswith("userID"): # header
        predictions.write(l)
        continue
    u,b = l.strip().split(',') # Read the user and item from the "pairs" file and write out your prediction
    prediction = alpha + beta_user.get(u, 0) + beta_book.get(b, 0)
    predictions.write(u + ',' + b + ',' + str(prediction) + '\n')
    
predictions.close()

NameError: name 'alpha' is not defined

In [448]:
f = open("answers_hw3.txt", 'w')
f.write(str(answers) + '\n')
f.close()